In [4]:
# import modules
import time
import os
import ee 
import geemap

In [3]:
#! pip install geemap

In [7]:
# Trigger the authentication flow.
#ee.Authenticate()
# Initialize the library.
ee.Initialize()

In [8]:
# initialize our map
myMap = geemap.Map()
myMap

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [11]:
# Draw aoi on map above or you can fetch your shapefile from feature collection 
#aoi = ee.FeatureCollection(myMap.draw_features).geometry()

# # get our Cameroon boundary
aoi = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME','Cameroon')).geometry()

# Sentinel-2 image filtered for 2019 
se2 = ee.ImageCollection('COPERNICUS/S2').filterDate("2019-01-01","2019-12-31").filterBounds(aoi).median().divide(10000)

rgb = ['B4','B3','B2']

# set some thresholds
rgbViz = {"min":0.0, "max":0.3,"bands":rgb}


# initialize our map
map1 = geemap.Map()
map1.centerObject(aoi, 7)
map1.addLayer(se2.clip(aoi), rgbViz, "S2")

map1.addLayerControl()
map1

Map(center=[5.68207224940031, 12.739010886474453], controls=(WidgetControl(options=['position', 'transparent_b…

In [12]:
# Function to mask clouds using the Sentinel-2 QA band

def se2mask(image):
    quality_band = image.select('QA60')
    
    # using the bit mask for clouds and cirrus clouds respectively
    cloudmask = 1 << 10
    cirrusmask = 1 << 11
    
    # we only want clear skies
    mask = quality_band.bitwiseAnd(cloudmask).eq(0) and (quality_band.bitwiseAnd(cirrusmask).eq(0))
    
    # we'll divide by 10000 to make interpreting the reflectance values easier
    return image.updateMask(mask).divide(10000)
    
se2 = ee.ImageCollection('COPERNICUS/S2').filterDate(
    "2019-01-01","2019-12-31").filterBounds(aoi).filter(
    ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",20)).map(se2mask).median()

# initialize our map
map2 = geemap.Map()
map2.centerObject(aoi, 7)
map2.addLayer(se2.clip(aoi), rgbViz, "S2")

map2.addLayerControl()
map2

Map(center=[5.68207224940031, 12.739010886474453], controls=(WidgetControl(options=['position', 'transparent_b…

In [ ]:
# specify the folder to download image
out_dir = os.path.join(os.path.expanduser('~'), 'your_location')
filename = os.path.join(out_dir, 'filename.tif')

In [ ]:
# clip image to your aoi and save 
image = se2.clip(aoi).unmask()
geemap.ee_export_image(image, filename=filename, scale=10, region=aoi, file_per_band=False)